# 🐶 Classification of Dog's Breed by Pytorch CNN Models 🐶

![](https://www.petsworld.in/blog/wp-content/uploads/2015/05/Golden-Retriever-Puppies.jpg)


For this image classification task, you will be using an image classification application using a deep learning model called a convolutional neural network (often abbreviated as CNN). CNNs work particularly well for detecting features in images like colors, textures, and edges; then using these features to identify objects in the images. You'll use a CNN that has already learned the features from a giant dataset of 1.2 million images called **ImageNet**. There are different types of CNNs that have different structures (architectures) that work better or worse depending on your criteria. With this project, you'll explore the three different architectures (AlexNet, VGG, and ResNet) and determine which is best for your application.

## Defining Functions necessary for this project

In [1]:
def get_input_args():
    """
    Arguments:
      1. Image Folder as dire with default value 'pet_images'
      2. CNN Model Architecture 
      3. Text File with Dog Names as dogfile with default value 'dognames.txt'
    This function returns these arguments as an dictionary.
    Parameters:
     None - simply using built-in function input to create & store arguments
    Returns:
     model_inputs - dictionary data structure that stores the argument values that user gives  
    """
    # Creating required dictionary for holding necessary path and model selection data
    directory = '../input/petimages/'
    dogfile = '../input/dognames/dognames.txt'
    try:
        model_selection = input("Which model would you like to use vgg, alexnet,or resnet: ")
        
    except:
        model_selection = "vgg"
        
    model_inputs = {'dire': directory,'dogfile':dogfile,'model':model_selection}
    
    return model_inputs

In [2]:
from os import listdir

def get_pet_labels(image_dir):
    """
    Creates a dictionary of pet labels (results_dic) based upon the filenames 
    of the image files. These pet image labels are used to check the accuracy 
    of the labels that are returned by the classifier function, since the 
    filenames of the images contain the true identity of the pet in the image.
    Be sure to format the pet labels so that they are in all lower case letters
    and with leading and trailing whitespace characters stripped from them.
    (ex. filename = 'Boston_terrier_02259.jpg' Pet label = 'boston terrier')
    Parameters:
     image_dir - The (full) path to the folder of images that are to be
                 classified by the classifier function (string)
    Returns:
      results_dic - Dictionary with 'key' as image filename and 'value' as a 
      List. The list contains for following item:
         index 0 = pet image label (string)
    """
    file_names = listdir(image_dir)
    results_dict = dict()
    
    for file in range(0,len(file_names),1):
        
        if file_names[file][0] != ".":
            word_list = file_names[file].lower().split("_")
      
            pet_label = ""
            for word in word_list:
               if word.isalpha():
                    pet_label += word + " "
            
            pet_label = pet_label.strip() 
            if file_names[file] not in results_dict:
                results_dict[file_names[file]] = [pet_label] 
            else:
               print("** Warning: Duplicate files exist in directory:", 
                     file_names[file])
        
    return results_dict

In [3]:
# Imports classifier function for using CNN to classify images 
def classify_images(images_dir, results_dic, model):
    """
    Creates classifier labels with classifier function, compares pet labels to 
    the classifier labels, and adds the classifier label and the comparison of 
    the labels to the results dictionary using the extend function. Be sure to
    format the classifier labels so that they will match your pet image labels.
    The format will include putting the classifier labels in all lower case 
    letters and strip the leading and trailing whitespace characters from them.
    For example, the Classifier function returns = 'Maltese dog, Maltese terrier, Maltese' 
    so the classifier label = 'maltese dog, maltese terrier, maltese'.
    Recall that dog names from the classifier function can be a string of dog 
    names separated by commas when a particular breed of dog has multiple dog 
    names associated with that breed. For example, you will find pet images of
    a 'dalmatian'(pet label) and it will match to the classifier label 
    'dalmatian, coach dog, carriage dog' if the classifier function correctly 
    classified the pet images of dalmatians.
     PLEASE NOTE: This function uses the classifier() function defined in the notebook.
     Parameters: 
      images_dir - The (full) path to the folder of images that are to be
                   classified by the classifier function (string)
      results_dic - Results Dictionary with 'key' as image filename and 'value'
                    as a List. Where the list will contain the following items: 
                  index 0 = pet image label (string)
                --- where index 1 & index 2 are added by this function ---
                  index 1 = classifier label (string)
                  index 2 = 1/0 (int)  where 1 = match between pet image
                    and classifer labels and 0 = no match between labels
      model - Indicates which CNN model architecture will be used by the 
              classifier function to classify the pet images,
              values must be either: resnet alexnet vgg (string)
     Returns:
           None - results_dic is mutable data type so no return needed.         
    """
    for imagename in results_dic:
        image = images_dir + imagename
        model_label = classifier(image,model)
        
        model_label = model_label.lower().strip()
        truth_value = results_dic[imagename][0]
        if truth_value in model_label:
            results_dic[imagename].extend([model_label,1])
        else:
            results_dic[imagename].extend([model_label,0])
        
    None

## transfer learning to get pre-trained models in pytorch framework

In [4]:
import ast
from PIL import Image
import torchvision.transforms as transforms
from torch.autograd import Variable
import torchvision.models as models
from torch import __version__

resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
vgg16 = models.vgg16(pretrained=True)

models = {'resnet': resnet18, 'alexnet': alexnet, 'vgg': vgg16}

# obtain ImageNet labels
with open('../input/imagenet1000clsidtohumans/imagenet1000_clsid_to_human.txt') as imagenet_classes_file:
    imagenet_classes_dict = ast.literal_eval(imagenet_classes_file.read())

def classifier(img_path, model_name):
    # load the image
    img_pil = Image.open(img_path)

    # define transforms
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # preprocess the image
    img_tensor = preprocess(img_pil)
    
    # resize the tensor (add dimension for batch)
    img_tensor.unsqueeze_(0)
    
    # wrap input in variable, wrap input in variable - no longer needed for
    # v 0.4 & higher code changed 04/26/2018 by Jennifer S. to handle PyTorch upgrade
    pytorch_ver = __version__.split('.')
    
    # pytorch versions 0.4 & hihger - Variable depreciated so that it returns
    # a tensor. So to address tensor as output (not wrapper) and to mimic the 
    # affect of setting volatile = True (because we are using pretrained models
    # for inference) we can set requires_gradient to False. Here we just set 
    # requires_grad_ to False on our tensor 
    if int(pytorch_ver[0]) > 0 or int(pytorch_ver[1]) >= 4:
        img_tensor.requires_grad_(False)
    
    # pytorch versions less than 0.4 - uses Variable because not-depreciated
    else:
        # apply model to input
        # wrap input in variable
        data = Variable(img_tensor, volatile = True) 

    # apply model to input
    model = models[model_name]

    # puts model in evaluation mode
    # instead of (default)training mode
    model = model.eval()
    
    # apply data to model - adjusted based upon version to account for 
    # operating on a Tensor for version 0.4 & higher.
    if int(pytorch_ver[0]) > 0 or int(pytorch_ver[1]) >= 4:
        output = model(img_tensor)

    # pytorch versions less than 0.4
    else:
        # apply data to model
        output = model(data)

    # return index corresponding to predicted class
    pred_idx = output.data.numpy().argmax()

    return imagenet_classes_dict[pred_idx]

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [5]:
def adjust_results4_isadog(results_dic, dogfile):
    """
    Adjusts the results dictionary to determine if classifier correctly 
    classified images 'as a dog' or 'not a dog' especially when not a match. 
    Demonstrates if model architecture correctly classifies dog images even if
    it gets dog breed wrong (not a match).
    Parameters:
      results_dic - Dictionary with 'key' as image filename and 'value' as a 
                    List. Where the list will contain the following items: 
                  index 0 = pet image label (string)
                  index 1 = classifier label (string)
                  index 2 = 1/0 (int)  where 1 = match between pet image
                    and classifer labels and 0 = no match between labels
                ------ where index 3 & index 4 are added by this function -----
                 index 3 = 1/0 (int)  where 1 = pet image 'is-a' dog and 
                            0 = pet Image 'is-NOT-a' dog. 
                 index 4 = 1/0 (int)  where 1 = Classifier classifies image 
                            'as-a' dog and 0 = Classifier classifies image  
                            'as-NOT-a' dog.
     dogfile - A text file that contains names of all dogs from the classifier
               function and dog names from the pet image files. This file has 
               one dog name per line dog names are all in lowercase with 
               spaces separating the distinct words of the dog name. Dog names
               from the classifier function can be a string of dog names separated
               by commas when a particular breed of dog has multiple dog names 
               associated with that breed (ex. maltese dog, maltese terrier, 
               maltese) (string - indicates text file's filename)
    Returns:
           None - results_dic is mutable data type so no return needed.
    """           
    dognames_dic = dict()
    with open(dogfile,'r') as f:
        for line in f:
            if line not in dognames_dic:            
                dognames_dic[line.rstrip()] = 1
            else:
                print("*Waring Duplicate dogname found in dognames text file Please revise the file*")
    for i in results_dic:
        if results_dic[i][0] in dognames_dic:
            results_dic[i].append(1)
        else:
            results_dic[i].append(0)
        
        if results_dic[i][1] in dognames_dic:
            results_dic[i].append(1)
        else:
            results_dic[i].append(0)     
    None

In [6]:
def calculates_results_stats(results_dic):
    """
    Calculates statistics of the results of the program run using classifier's model 
    architecture to classifying pet images. Then puts the results statistics in a 
    dictionary (results_stats_dic) so that it's returned for printing as to help
    the user to determine the 'best' model for classifying images. Note that 
    the statistics calculated as the results are either percentages or counts.
    Parameters:
      results_dic - Dictionary with key as image filename and value as a List 
             (index)idx 0 = pet image label (string)
                    idx 1 = classifier label (string)
                    idx 2 = 1/0 (int)  where 1 = match between pet image and 
                            classifer labels and 0 = no match between labels
                    idx 3 = 1/0 (int)  where 1 = pet image 'is-a' dog and 
                            0 = pet Image 'is-NOT-a' dog. 
                    idx 4 = 1/0 (int)  where 1 = Classifier classifies image 
                            'as-a' dog and 0 = Classifier classifies image  
                            'as-NOT-a' dog.
    Returns:
     results_stats_dic - Dictionary that contains the results statistics (either
                    a percentage or a count) where the key is the statistic's 
                     name (starting with 'pct' for percentage or 'n' for count)
                     and the value is the statistic's value. See comments above
                     and the previous topic Calculating Results in the class for details
                     on how to calculate the counts and statistics.
    """        
    
    results_stats_dic = dict()
    results_stats_dic['n_dogs_img'] = 0
    results_stats_dic['n_match'] = 0
    results_stats_dic['n_correct_dogs'] = 0
    results_stats_dic['n_correct_notdogs'] = 0
    results_stats_dic['n_correct_breed'] = 0    
    
    results_stats_dic['n_images'] = len(results_dic)
    
    for key in results_dic:
        if results_dic[key][3] == 1:
            results_stats_dic['n_dogs_img'] += 1
            if results_dic[key][4] == 1:
                results_stats_dic['n_correct_dogs'] += 1
        else:
            if results_dic[key][4] == 0:
                results_stats_dic['n_correct_notdogs'] += 1
            
        if results_dic[key][2] == 1:
            results_stats_dic['n_match'] += 1
            
        if results_dic[key][2] == 1 and results_dic[key][3] == 1:
                results_stats_dic['n_correct_breed'] += 1
   
    results_stats_dic['n_notdogs_img'] = results_stats_dic['n_images'] - results_stats_dic['n_dogs_img']
    # Calculates % correct for matches
    results_stats_dic['pct_match'] = (results_stats_dic['n_match']/results_stats_dic['n_images'])*100
    
     # Calculates % correct breed of dog
    results_stats_dic['pct_correct_breed'] = (results_stats_dic['n_correct_breed']/results_stats_dic['n_dogs_img'])*100  
    
    # Calculates % correct dogs
    results_stats_dic['pct_correct_dogs'] = (results_stats_dic['n_correct_dogs']/results_stats_dic['n_dogs_img'])*100 
    
    # Calculates % correct not-a-dog images
    if results_stats_dic['n_notdogs_img'] > 0:
        results_stats_dic['pct_correct_notdogs'] = (results_stats_dic['n_correct_notdogs'] /
                                                results_stats_dic['n_notdogs_img'])*100.0
    else:
        results_stats_dic['pct_correct_notdogs'] = 0.0


    return results_stats_dic

In [7]:
def print_results(results_dic, results_stats_dic, model, 
                  print_incorrect_dogs = False, print_incorrect_breed = False):
    """
    Prints summary results on the classification and then prints incorrectly 
    classified dogs and incorrectly classified dog breeds if user indicates 
    they want those printouts (use non-default values)
    Parameters:
      results_dic - Dictionary with key as image filename and value as a List 
             (index)idx 0 = pet image label (string)
                    idx 1 = classifier label (string)
                    idx 2 = 1/0 (int)  where 1 = match between pet image and 
                            classifer labels and 0 = no match between labels
                    idx 3 = 1/0 (int)  where 1 = pet image 'is-a' dog and 
                            0 = pet Image 'is-NOT-a' dog. 
                    idx 4 = 1/0 (int)  where 1 = Classifier classifies image 
                            'as-a' dog and 0 = Classifier classifies image  
                            'as-NOT-a' dog.
      results_stats_dic - Dictionary that contains the results statistics (either
                   a  percentage or a count) where the key is the statistic's 
                     name (starting with 'pct' for percentage or 'n' for count)
                     and the value is the statistic's value 
      model - Indicates which CNN model architecture will be used by the 
              classifier function to classify the pet images,
              values must be either: resnet alexnet vgg (string)
      print_incorrect_dogs - True prints incorrectly classified dog images and 
                             False doesn't print anything(default) (bool)  
      print_incorrect_breed - True prints incorrectly classified dog breeds and 
                              False doesn't print anything(default) (bool) 
    Returns:
           None - simply printing results.
    """    
    # Printing summary statistics 
    print("\n\n*** Results Summary for CNN Model Architecture",model.upper(), 
          "***")
    print("{:20}: {:3d}".format('N Images', results_stats_dic['n_images']))
    print("{:20}: {:3d}".format('N Dog Images', results_stats_dic['n_dogs_img']))

   
    print("{:20}: {:3d}".format('N Not-Dog Images', results_stats_dic['n_notdogs_img']))


    # Prints summary statistics (percentages) on Model Run
    print(" ")
    for key in results_stats_dic:
        if 'p' in key:
            s = key.title().split("_")
            s[0] = "Percentage of" 
            processed = ""
            for i in s:
                processed += "{} ".format(i)
            print("{:20}: {:3f}".format(processed, results_stats_dic[key]))


    # IF print_incorrect_dogs == True AND there were images incorrectly 
    # classified as dogs or vice versa - print out these cases
    if (print_incorrect_dogs and 
        ( (results_stats_dic['n_correct_dogs'] + results_stats_dic['n_correct_notdogs'])
          != results_stats_dic['n_images'] ) 
       ):
        print("\nINCORRECT Dog/NOT Dog Assignments:")

        # process through results dict, printing incorrectly classified dogs
        for key in results_dic:
            if ( sum(results_dic[key][3:]) == 2 and results_dic[key][2] == 0 ):
                print("Real: {:>26}   Classifier: {:>30}".format(results_dic[key][0], results_dic[key][1]))

    # IF print_incorrect_breed == True AND there were dogs whose breeds 
    # were incorrectly classified - print out these cases                    
    if (print_incorrect_breed and 
        (results_stats_dic['n_correct_dogs'] != results_stats_dic['n_correct_breed']) 
       ):
        print("\nINCORRECT Dog Breed Assignment:")

        # process through results dict, printing incorrectly classified breeds
        for key in results_dic:

            # Pet Image Label is-a-Dog, classified as-a-dog but is WRONG breed
            if ( sum(results_dic[key][3:]) == 2 and
                results_dic[key][2] == 0 ):
                print("Real: {:>26}   Classifier: {:>30}".format(results_dic[key][0],
                                                          results_dic[key][1]))

## This is where everythings boil downs to Main fucntion

In [8]:
# Imports python modules
from time import time, sleep
# Main program function defined below
def main():
    
    # Measures total program runtime by collecting start time
    start_time = time()
    
    # Getting Inputs from the user
    in_arg = get_input_args()
   
    # This function creates the results dictionary that contains the results, 
    # this dictionary is returned from the function call as the variable results
    results = get_pet_labels(in_arg['dire'])
    
    # Creates Classifier Labels with classifier function, Compares Labels, 
    # and adds these results to the results dictionary - results
    classify_images(in_arg['dire'], results, in_arg['model'])

    # Adjusts the results dictionary to determine if classifier correctly 
    # classified images as 'a dog' or 'not a dog'. This demonstrates if 
    # model can correctly classify dog images as dogs (regardless of breed)
    adjust_results4_isadog(results, in_arg['dogfile'])

    # This function creates the results statistics dictionary that contains a
    # summary of the results statistics (this includes counts & percentages). This
    # dictionary is returned from the function call as the variable results_stats    
    # Calculates results of run and puts statistics in the Results Statistics
    # Dictionary - called results_stats
    results_stats = calculates_results_stats(results)


    # Prints summary results, incorrect classifications of dogs (if requested)
    # and incorrectly classified breeds (if requested)
    print_results(results, results_stats, in_arg['model'], True, True)
    
    # Measure total program runtime by collecting end time
    end_time = time()
    
    # Computes overall runtime in seconds & prints it in hh:mm:ss format
    tot_time = end_time - start_time#calculate difference between end time and start time
    print("\n** Total Elapsed Runtime:",
          str(round((tot_time/3600)))+":"+str(round((tot_time%3600)/60))+":"
          +str(round((tot_time%3600)%60))+" **")
    

# Call to main function to run the program
if __name__ == "__main__":
    main()



*** Results Summary for CNN Model Architecture VGG ***
N Images            :  40
N Dog Images        :  30
N Not-Dog Images    :  10
 
Percentage of Match : 87.500000
Percentage of Correct Breed : 93.333333
Percentage of Correct Dogs : 100.000000
Percentage of Correct Notdogs : 100.000000

INCORRECT Dog Breed Assignment:
Real:                     beagle   Classifier:  walker hound, walker foxhound
Real:             great pyrenees   Classifier:                         kuvasz

** Total Elapsed Runtime: 0:0:13 **


## Model Performances

### 1. VGG
Which model would you like to use vgg, alexnet,or resnet:  vgg

*** Results Summary for CNN Model Architecture VGG ***

N Images            :  40<br>
N Dog Images        :  30<br>
N Not-Dog Images    :  10
 
Percentage of Match : 87.500000<br>
Percentage of Correct Breed : 93.333333<br>
Percentage of Correct Dogs : 100.000000<br>
Percentage of Correct Notdogs : 100.000000

|INCORRECT Dog Breed Assignment||
|---|---|
|Real:                     beagle  | Classifier:  walker hound, walker foxhound |
|Real:             great pyrenees  | Classifier:                         kuvasz |

**Total Elapsed Runtime: 0:0:16**

### 2. ALEXNET

Which model would you like to use vgg, alexnet,or resnet:  alexnet


*** Results Summary for CNN Model Architecture ALEXNET ***

N Images            :  40<br>
N Dog Images        :  30<br>
N Not-Dog Images    :  10<br>
 
Percentage of Match : 75.000000<br>
Percentage of Correct Breed : 80.000000<br>
Percentage of Correct Dogs : 100.000000<br>
Percentage of Correct Notdogs : 100.000000<br>

|INCORRECT Dog Breed Assignment||
|---|---|
|Real:             boston terrier |  Classifier:                        basenji |
|Real:           golden retriever |  Classifier:           afghan hound, afghan |
|Real:                     beagle |  Classifier:               english foxhound |
|Real:                     beagle |  Classifier:  walker hound, walker foxhound |
|Real:             great pyrenees |  Classifier:                         kuvasz |
|Real:           golden retriever |  Classifier:                tibetan mastiff |

**Total Elapsed Runtime: 0:0:6**

### 3. RESNET

Which model would you like to use vgg, alexnet,or resnet:  resnet


*** Results Summary for CNN Model Architecture RESNET ***


N Images            :  40<br>
N Dog Images        :  30<br>
N Not-Dog Images    :  10<br>
 
Percentage of Match : 82.500000<br>
Percentage of Correct Breed : 90.000000<br>
Percentage of Correct Dogs : 100.000000<br>
Percentage of Correct Notdogs : 90.000000<br>

|INCORRECT Dog/NOT Dog Assignments||
|---|---|
|Real:                     beagle   | Classifier:  walker hound, walker foxhound |
|Real:             great pyrenees   | Classifier:                         kuvasz |
|Real:           golden retriever   | Classifier:                       leonberg |

|INCORRECT Dog Breed Assignment||
|---|---|
|Real:                     beagle  | Classifier:  walker hound, walker foxhound |
|Real:             great pyrenees  | Classifier:                         kuvasz |
|Real:           golden retriever  | Classifier:                       leonberg |

**Total Elapsed Runtime: 0:0:6**

## <center>Model evaluation

| Model | Percentage of Match | Percentage of Correct Breed | Percentage of Correct Dogs | Percentage of Correct Notdogs |
    |---|---|---|---|---|
| Vgg   | 87.50% | 93.33% | 100% | 100% |
| Alexnet   | 75% | 80% | 100% | 100% |
| Resnet   | 82.50% | 90% | 100% | 90% |


# <center>I hope you enjoyed this project, Please do Upvote😊 and Share your thoughts i would love to hear from the community🙏